
# 0.初始化OpenAI客户端
目前大多数大模型对遵循了OpenAI提供的大模型接口规范，因此这里直接使用OpenAI提供的SDK库。

国内大模型大多也都兼容OpenAI的SDK，如果你无法访问GPT模型，可以选择国内的模型来使用。

这里以阿里千万模型为例，使用步骤如下：
1. 注册并阿里云百炼平台，地址：https://bailian.console.aliyun.com/#/home
2. 登录后，查看自己的api_key，地址：https://bailian.console.aliyun.com/?apiKey=1#/api-key
3. 将资料中的.env文件中的OPENAI_API_KEY改成自己的KEY
4. 用VSCODE打开code目录中的ai-programing.ipynb文件，会提示安装Jupyter插件，安装后即可运行Python代码
5. 注意，如果修改了.env文件，需要重启VSCODE，并修改代码才能生效

In [16]:
# 首先需要安装一些基本环境
%pip install --upgrade openai
%pip install pdfplumber



In [38]:

from openai import OpenAI
import json

# 读取本地的 .env 文件到环境变量，在 .env 文件中配置 OPENAI_API_KEY 和 OPENAI_BASE_URL
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务，会自动从环境变量加载 OPENAI_API_KEY 和 OPENAI_BASE_URL
client = OpenAI() 

# 模型名称
model = "qwen-max"

# 定义方法，向大模型发送消息
def get_completion(prompt, model = model, temperature = 0):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 结果的随机性
    )
    return response.choices[0].message.content

print(client.base_url)

https://dashscope.aliyuncs.com/compatible-mode/v1/


# 1.普通Prompt问答
特征：
- 利用大模型推理能力完成应用端无法实现或难以实现的功能

应用场景：
- 文本摘要
- 舆情分析
- 坐席检查

## 1.1.文本摘要
例如，我们可以使用大模型来生成一篇文章的摘要，或者从一篇文章中提取最重要的部分。

In [117]:
# text是一段提示词编写的最佳实践方案，我们通过Prompt问答的方式让大模型帮我们抽取文本摘要，而且是用中文
text = f"""
    You should express what you want a model to do by \
    providing instructions that are as clear and \
    specific as you can possibly make them. \
    This will guide the model towards the desired output, \
    and reduce the chances of receiving irrelevant \
    or incorrect responses.Don't confuse writing a \
    clear prompt with writing a short prompt. \
    In many cases,longer prompts provide more clarityand \
    context for the model, which can lead to \
    more detailed and relevant outputs.
"""

# 提示词：将```中的文本提取为一句话的摘要，限定回答内容不超过50个单词并且以中文回答
#prompt = f"""
#Summarize the text delimited by triple backticks into a single sentence, \
#limit your response to 50 words and translate your answer to Chinese.
# ```{text}```
# """

prompt = f"""
将```中的文本提取为一句话的摘要，限定回答内容不超过10个词，用中文回答
```{text}```
"""
response= get_completion(prompt)
print(response)

清楚、具体地指导模型，确保输出相关且准确。


## 1.2.舆情分析
对于电商网站来说，用户对于商品的评价是非常重要的分析数据：

- 可以从评价的文本中分析出哪些词是正面的，哪些词是负面的，正面和负面评价的情感比例是多少？

- 可以从评价文本中提取对商品的负面评价的因素，针对性的给出回复，也可以作为商家改进的方向。


但是商品的评价数量数以亿记，全靠人类去判断评价太复杂了，传统编程也难以分析文本中的感情因素，这个时候就需要利用大模型了。

In [119]:
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason(call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn't look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it(example,I crush \
very hard items like beans,ice,rice,etc.in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer floyr, and use the cross cutting blade \
first when making smcthies,then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables(if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet,use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year,the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales, Got it in about \
two days .
"""

In [120]:
# prompt = f"""
# You are a customer service AI assistant. \
# Your task is to send an email reply to a valued customer. \
# Given the customer email delimited by ```, \
# Generate a reply to thank the customer for their review. \
# If the sentiment is positive or neutral, thank them fortheir review. \
# If the sentiment is negative,apologize and suggest that \
# they can reach out to customer service. \
# Make sure to use specific details from the review. \
# Write in a concise and professional tone. \
# Sign the email as `AI customer agent`. \
# Customer review:```{review}```
# """

prompt = f"""
你是一名客服人工智能助理。\
你的任务是给一个重要的客户回复信息。\
给定以```分隔的客户的评价, 生成回复以感谢客户的评价。\
如果用户的评价是正面的或中性的，那就感谢他们的评价。\
如果用户评价情绪是负面的，道歉并建议他们可以联系客服 customerService@itcast.cn。\
确保使用审查中的具体细节。用简洁和专业的语气写作。用中文回复。\
将你的回复签名为"AI客户代理"。\
客户评价：```{review}```
"""


response= get_completion(prompt, temperature=0.7)
print(response)

尊敬的客户，

非常感谢您花时间分享您的购物体验。我们很高兴您能在11月享受到我们的季节性优惠，关于价格波动的问题，我们会向相关部门反馈并审视我们的定价策略，确保为所有顾客提供公平合理的价格。

对于您提到的产品质量细节，特别是底座及刀片锁定部位的设计变化，您的反馈是我们改进产品的宝贵依据。我们深感歉意给您带来的不便，如果您在使用过程中遇到任何问题，欢迎随时联系我们客服邮箱customerService@itcast.cn，我们将竭诚为您提供解决方案。

关于您细腻的使用技巧分享，我们十分感谢，并相信这将对其他用户也有很大的帮助。对于您遇到的电机异响问题以及保修期满的情况，我们感到遗憾。我们正不断努力提升产品耐用度与服务质量，以更好满足顾客的期待。

再次感谢您的中肯评价和长期支持，我们诚挚希望未来能以更优质的产品和服务回馈您的信任。

祝您生活愉快！

AI客户代理



# 2.Function Calling
示意图：<br/>

<img src="./fc.png" />


特征：
- 将应用端业务能力与AI大模型推理能力结合，简化复杂业务功能开发
- 利用应用端能力将AI与外部世界连接起来，弥补AI模型缺陷：
    - 大模型不是无所不知
    - 大模型知识往往比较落后
    - 大模型逻辑是统计规律，不是真逻辑，存在幻觉


应用场景：
- 智能体（旅行指南、日程安排等）
- 数据提取
- 数据聚合分析


In [121]:
def print_json(data):
    """
    打印参数。如果参数是有结构的（如字典或列表），则以格式化的 JSON 形式打印；
    否则，直接打印该值。
    """
    if hasattr(data, 'model_dump_json'):
        data = json.loads(data.model_dump_json())

    if (isinstance(data, (list))):
        for item in data:
            print_json(item)
    elif (isinstance(data, (dict))):
        print(json.dumps(
            data,
            indent=4,
            ensure_ascii=False
        ))
    else:
        print(data)

In [122]:
#  描述数据库表结构
users_schema = """
CREATE TABLE users (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    user_name VARCHAR(255) NOT NULL, -- 用户名，不允许为空
    email VARCHAR(255) UNIQUE, -- 邮箱，唯一
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP -- 注册时间，默认为当前时间
);
"""
items_schema = """
CREATE TABLE items (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    name VARCHAR(255) NOT NULL, -- 商品名称，不允许为空
    price DECIMAL(10,1) NOT NULL -- 价格，不允许为空
);
"""
orders_schema = """
CREATE TABLE orders (
    id LONG PRIMARY KEY NOT NULL, -- 主键，不允许为空
    user_id INT NOT NULL, -- 客户ID，不允许为空
    item_id STR NOT NULL, -- 商品ID，不允许为空
    price DECIMAL(10,1) NOT NULL, -- 下单价格，不允许为空
    amount INT NOT NULL, -- 下单数量，不允许为空
    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0-未支付; 1-已支付; 2-已取消; 3-已发货; 4-已完成; 5-已退款
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间
    pay_time TIMESTAMP, -- 支付时间，可以为空
    cancel_time TIMESTAMP, -- 取消时间，可以为空
    deliver_time TIMESTAMP, -- 发货时间，可以为空
    finish_time TIMESTAMP, -- 完成时间，可以为空
    refund_time TIMESTAMP -- 退款时间，可以为空
);
"""
database_schema_string = users_schema +"\n"+ items_schema +"\n"+ orders_schema

In [123]:
import sqlite3

# 创建数据库连接
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# 创建orders表
cursor.execute(users_schema)
cursor.execute(items_schema)
cursor.execute(orders_schema)

# 插入5条明确的模拟数据
mock_order_data = [
    (1, 1001, 11001, 48.8, 1, 0, '2024-06-12 10:00:00', None, None, None, None, None),
    (2, 1001, 11002, 79.9, 2, 1, '2024-06-16 11:00:00', '2024-06-16 11:02:00', None, None, None, None),
    (3, 1002, 21001, 25.0, 1, 4, '2024-06-17 12:30:00', '2024-06-17 12:35:00', None, '2024-06-17 16:30:00', '2024-06-20 20:35:00', None),
    (4, 1003, 21002, 64.8, 1, 1, '2024-06-20 14:00:00', '2024-06-20 14:01:00', None, None, None, None),
    (5, 1002, 31001, 89.9, 1, 2, '2024-06-28 18:20:00', None, '2024-06-28 18:50:00', None, None, None)
]

for record in mock_order_data:
    cursor.execute('''
    INSERT INTO orders (id, user_id, item_id, price, amount, status, create_time, pay_time, cancel_time, deliver_time, finish_time, refund_time)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', record)

mock_user_data = [
    (1001, 'Jack', 'jack@itcast.cn','2024-1-12 10:00:00'),
    (1002, 'Rose', 'rose@itcast.cn','2023-11-22 12:00:00'),
    (1003, 'Amy', 'amy@itcast.cn','2023-6-21 09:00:00'),
]

for record in mock_user_data:
    cursor.execute('''
    INSERT INTO users (id, user_name, email, create_time)
    VALUES (?, ?, ?, ?)
    ''', record)

mock_item_data = [
    (11001, '鼠标', 48.8),
    (11002, '键盘', 79.9),
    (21001, '五常大米2.5KG', 32.5),
    (21002, '五常大米5KG', 64.8),
    (31001, '金鱼调和油', 89.9),
]

for record in mock_item_data:
    cursor.execute('''
    INSERT INTO items (id, name, price)
    VALUES (?, ?, ?)
    ''', record)

# 提交事务
connection.commit()

In [124]:
# 以下参考官方案例：
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb


# 定义一个查询数据库的函数，传入SQL语句，返回查询结果
def query_db(sql):
    cursor.execute(sql)
    result = cursor.fetchall()
    return result

# 定义一个函数，调用OpenAI接口，传入消息体， 返回SQL语句
def get_sql_completion(messages, model = model):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 结果的随机性设置为0，因为SQL必须准确
        tools = [{ # 定义函数，供大模型调用
            "type": "function",
            "function": {
                "name": "query_db",
                "description": "Use this function to answer user questions about business. \
                            Input should be a fully formed SQL query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sql": {
                            "type": "string",
                            "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                The query should only contain grammars supported by SQLite.
                            """,
                        }
                    },
                    "required": ["sql"],
                }
            }
        }]
    )
    return response.choices[0].message

In [126]:
# prompt = "统计2024年6月的销售额"
prompt = "统计每月每件商品的销售额"
# prompt = "哪个用户消费最高？消费了多少？"

messages = [
    {"role": "user", "content": prompt}
]
# 第一次调用，大模型回返回函数调用的信息
response = get_sql_completion(messages)

# 判断是否正确返回，如果没有返回则置空
if response.content is None:
    response.content = ""

# 大模型返回的函数调用信息需要追加到message中，作为下一次调用的上下文
messages.append(response)
print("====Function Calling====")
print_json(response)

# 如果返回了函数调用信息，则需要应用端调用函数，获取函数的返回值
if response.tool_calls is not None:
    tool_call = response.tool_calls[0]
    # 获取函数名称，执行对应函数，因为复杂业务中可以能有有多个函数，所以需要根据函数名称判断
    if tool_call.function.name == "query_db":
        arguments = tool_call.function.arguments
        args = json.loads(arguments)
        print("====SQL====")
        print(args["sql"])
        # 根据大模型生成的SQL，查询数据库
        result = query_db(args["sql"])
        print("====DB Records====")
        print(result)

        # 将数据库查询结果追加到messages中，作为下次prompt的上下文
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "query_db",
            "content": str(result)
        })

        # 再次调用大模型，获取最终回复
        response = get_sql_completion(messages)
        print("====最终回复====")
        print(response.content)

====Function Calling====
{
    "content": "",
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "",
            "function": {
                "arguments": "{\"sql\": \"SELECT strftime('%Y-%m', orders.create_time) AS month, items.name AS product, SUM(orders.price * orders.amount) AS sales\\nFROM orders\\nJOIN items ON orders.item_id = items.id\\nGROUP BY month, product\"}",
                "name": "query_db"
            },
            "type": "function"
        }
    ]
}
====SQL====
SELECT strftime('%Y-%m', orders.create_time) AS month, items.name AS product, SUM(orders.price * orders.amount) AS sales
FROM orders
JOIN items ON orders.item_id = items.id
GROUP BY month, product
====DB Records====
[('2024-06', '五常大米2.5KG', 25), ('2024-06', '五常大米5KG', 64.8), ('2024-06', '金鱼调和油', 89.9), ('2024-06', '键盘', 159.8), ('2024-06', '鼠标', 48.8)]
====最终回复====
以下是2024年6月每件商品的销售额统计：

- **五常大米2.5KG**：销售额为25元
- **五常大米5KG**：销售额为64.8元
- **金鱼调和油**：销售额为89.9元
-

# 3.RAG
<strong>示意图：</strong>
<img src="./rag.png" />

<strong>离线步骤：</strong>
- 文档加载
- 文档切分
- 向量化
- 灌入向量数据库


<strong>在线步骤：</strong>
- 获得用户问题
- 用户问题向量化
- 检索向量数据库
- 将检索结果和用户问题填入Prompt模版
- 用最终获得的Prompt调用LLM
- 由 LLM 生成回复

<strong>应用场景：</strong>
- AI客服
- AI知识库





In [114]:
prompt = "论语中教育的目的是什么"

response= get_completion(prompt)
print(response)

《论语》是儒家经典之一，记载了孔子及其弟子的言行。在《论语》中，教育的目的可以概括为以下几个方面：

1. **培养君子**：《论语》强调教育的首要目的是培养“君子”，君子是指具有高尚道德品质、学问修养和社会责任感的人。孔子认为，君子应当仁爱、礼让、诚信、智慧，并能以身作则，引领社会风气。

2. **修身齐家治国平天下**：这是儒家教育的理想路径。首先通过教育修身（提升个人品德与能力），进而管理好家庭（齐家），然后有能力参与国家治理（治国），最终达到社会和谐与世界太平（平天下）。这一理念体现了从个人修养到社会责任的递进过程。

3. **传道授业解惑**：教育不仅要传授知识和技能（授业），更重要的是传播道德观念和人生哲理（传道），以及解答疑惑（解惑），帮助学生形成正确的世界观和价值观。

4. **知行合一**：孔子提倡“知之为知之，不知为不知”，强调诚实求知的态度，同时注重理论与实践相结合，倡导“学而时习之”，通过实践来检验和深化所学知识，实现知识与行动的统一。

5. **仁爱教育**：仁是儒家的核心概念，教育的目的是培养人的仁爱心，使个体能够爱人、尊重人，以仁爱之心处理人际关系，构建和谐社会。

总之，《论语》中的教育目的不仅仅是知识的传授，更重视德性的培养、人格的完善以及社会责任感的建立，旨在培养出既有深厚文化素养，又能积极贡献于社会的全面发展的人才。


In [127]:
import pdfplumber
import re
# 定义函数，提取PDF文件内容
def extract_text_from_pdf(filename, page_numbers=None):
    full_text = ''
    paragraphs = []
    # 提取全部文本
    with pdfplumber.open('中二知识笔记.pdf') as pdf:
        for i, page in enumerate(pdf.pages):
            if page_numbers is not None and i not in page_numbers:
                continue
            text = page.extract_text()
            full_text += text
    chapters = re.split('（[\u4e00-\u9fa5]）', full_text)
    for chapter in chapters:
        paragraphs.append(chapter)
    return paragraphs

In [128]:
paragraphs = extract_text_from_pdf("中二知识笔记.pdf", [1])


In [101]:
# 输出每段话
for paragraph in paragraphs:
    print(paragraph + "\n")



孔子及《论语》主要思想
角度 观点
教育作用 庶、富、教。性相近，习相远。
教育对象 “有教无类”；教育民主思想。
教育目的 以完善人格为教育的首要目的，培养士和君子。
教学内容 文、行、忠、义。
教学过程 学、思、习、行。
教学原则 因材施教、启发诱导、学思结合、谦虚笃实
教师观 其身正，不令而行；其身不正，虽令不从


孟子主要思想
角度 观点
人性论 人性本善，人先天具有仁、义、礼、智四个“善端”。
教育作用 发扬善端，培养道德完人，得天下英才而教育之。
教学原则 循序渐进，专心有恒


荀子主要思想
角度 观点
人性论 性恶论，化性起伪，善德是后天习得的。重视教育的作用。
教学原则 学以致用，锲而不舍


希腊三贤
教育家 思想
苏格拉底 产婆术（谈话法），分三步：讽刺—定义—助产术。国外启发式教育第一人。
柏拉图 《理想国》提出普及教育的主张，教育目的是培养哲学王。
亚里士多德 提出自由教育，提倡对学生进行和谐全面发展的教育。灵魂说。



In [32]:
# 准备环境，这里我们采用阿里百炼提供的向量模型
# %pip install dashscope

In [129]:
# 文本向量化
import dashscope
from http import HTTPStatus
dashscope.api_key=client.api_key

def embed_with_str(text):
    resp = dashscope.TextEmbedding.call(
        model=dashscope.TextEmbedding.Models.text_embedding_v3,
        input=text)
    if resp.status_code == HTTPStatus.OK:
        return [x['embedding'] for x in resp.output['embeddings']]
    else:
        return []

In [72]:
test_query = [
    '衣服的质量杠杠的，很漂亮，不枉我等了这么久啊，喜欢，以后还来这里买'
]
vecs = embed_with_str(test_query)[0]
print(vecs[:10])

[-0.06612710654735565, 0.010652517899870872, -0.08436639606952667, 0.0046180314384400845, -0.015474286861717701, -0.010749535635113716, -0.03143366053700447, 0.06787342578172684, -0.03859354928135872, 0.05378648638725281]


文本向量化以后，我们可以利用向量间距离来计算文本相似度。
有两种不同的计算方式：
1. 欧氏距离
2. 余弦距离

<img src="sim.png" style="margin-left: 0px" width=500px>

In [68]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    '''余弦距离 -- 越大越相似'''
    return dot(a, b)/(norm(a)*norm(b))


def l2(a, b):
    '''欧式距离 -- 越小越相似'''
    x = np.asarray(a)-np.asarray(b)
    return norm(x)

In [74]:
# query = "国际争端"

# 且能支持跨语言
query = "global conflicts"

documents = [
    "联合国就以色列空袭加沙致平民伤亡事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验",
]

query_vec = embed_with_str([query])[0]
doc_vecs = embed_with_str(documents)

print("Cosine distance:")
print(cos_sim(query_vec, query_vec))
for vec in doc_vecs:
    print(cos_sim(query_vec, vec))

print("\nEuclidean distance:")
print(l2(query_vec, query_vec))
for vec in doc_vecs:
    print(l2(query_vec, vec))

Cosine distance:
0.9999999999999999
0.5491419200342438
0.41206917395007464
0.4243735945985427
0.3256978300146348
0.3413411629074793

Euclidean distance:
0.0
0.9495871790553803
1.0843713874148921
1.072964301120678
1.1612943820434174
1.1477441864104283


In [76]:
# 导入一个向量数据库
# %pip install chromadb

In [130]:
import chromadb
from chromadb.config import Settings


class MyVectorDBConnector:
    def __init__(self, collection_name, embedding_fn):
        chroma_client = chromadb.Client(Settings(allow_reset=True))

        # 为了演示，实际不需要每次 reset()
        chroma_client.reset()

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(name=collection_name)
        self.embedding_fn = embedding_fn

    def add_documents(self, documents):
        '''向 collection 中添加文档与向量'''
        self.collection.add(
            embeddings=self.embedding_fn(documents),  # 每个文档的向量
            documents=documents,  # 文档的原文
            ids=[f"id{i}" for i in range(len(documents))]  # 每个文档的 id
        )

    def search(self, query, top_n):
        '''检索向量数据库'''
        results = self.collection.query(
            query_embeddings=self.embedding_fn([query]),
            n_results=top_n
        )
        return results

In [131]:
# 创建一个向量数据库对象
vector_db = MyVectorDBConnector("demo", embed_with_str)
# 向向量数据库中添加文档
vector_db.add_documents(paragraphs)

In [109]:
user_query = "希腊三贤"
results = vector_db.search(user_query, 2)

In [110]:
for para in results['documents'][0]:
    print(para+"\n")

希腊三贤
教育家 思想
苏格拉底 产婆术（谈话法），分三步：讽刺—定义—助产术。国外启发式教育第一人。
柏拉图 《理想国》提出普及教育的主张，教育目的是培养哲学王。
亚里士多德 提出自由教育，提倡对学生进行和谐全面发展的教育。灵魂说。

荀子主要思想
角度 观点
人性论 性恶论，化性起伪，善德是后天习得的。重视教育的作用。
教学原则 学以致用，锲而不舍




In [111]:

# 定义函数，实现知识检索
def knowledge_retrieval(query):
    # 1.检索向量数据库
    search_results = vector_db.search(query, 2)
    # 2.构建Prompt
    prompt = f"""
    你是一个问答机器人。
    你的任务是根据下述给定的已知信息回答用户问题。
    确保你的回复完全依据下述已知信息。不要编造答案。
    如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

    已知信息:
    {search_results}

    用户问：
    {query}

    请用中文回答用户问题。
    """

    # 3.调用LLM
    response = get_completion(prompt)
    return response

In [132]:
# user_query = "希腊三贤有哪几个？"
# user_query = "荀子的主要思想？"
user_query = "论语中教育的目的是什么？"

response = knowledge_retrieval(user_query)

print(response)

论语中教育的首要目的是以完善人格为教育的目标，旨在培养士和君子。
